# Planning
#### Chapters 10-11
----

This notebook serves as supporting material for topics covered in **Chapter 10 - Classical Planning** and **Chapter 11 - Planning and Acting in the Real World** from the book *[Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu)*. 
This notebook uses implementations from the [planning.py](https://github.com/aimacode/aima-python/blob/master/planning.py) module. 
See the [intro notebook](https://github.com/aimacode/aima-python/blob/master/intro.ipynb) for instructions.

We'll start by looking at `PDDL` and `Action` data types for defining problems and actions. 
Then, we will see how to use them by trying to plan a trip from *Sibiu* to *Bucharest* across the familiar map of Romania, from [search.ipynb](https://github.com/aimacode/aima-python/blob/master/search.ipynb) 
followed by some common planning problems and methods of solving them.

Let's start by importing everything from the planning module.

In [1]:
from planning import *
from notebook import psource

## CONTENTS

- PDDL
- Action
- Planning Problems
    * Air cargo problem
    * Spare tire problem
    * Three block tower problem
    * Shopping Problem
    * Cake problem
- Solving Planning Problems
    * GraphPlan

## PDDL

PDDL stands for Planning Domain Definition Language.
The `PDDL` class is used to represent planning problems in this module. The following attributes are essential to be able to define a problem:
* an initial state
* a set of goals
* a set of viable actions that can be executed in the search space of the problem

View the source to see how the Python code tries to realise these.

In [2]:
psource(PDDL)

The `init` attribute is an expression that forms the initial knowledge base for the problem.
<br>
The `goals` attribute is an expression that indicates the goals to be reached by the problem.
<br>
Lastly, `actions` contains a list of `Action` objects that may be executed in the search space of the problem.
<br>
The `goal_test` method checks if the goal has been reached.
<br>
The `act` method acts out the given action and updates the current state.
<br>


## ACTION

To be able to model a planning problem properly, it is essential to be able to represent an Action. Each action we model requires at least three things:
* preconditions that the action must meet
* the effects of executing the action
* some expression that represents the action

The module models actions using the `Action` class

In [3]:
psource(Action)

This class represents an action given the expression, the preconditions and its effects. 
A list `precond` stores the preconditions of the action and a list `effect` stores its effects.
Negative preconditions and effects are input using a `~` symbol before the clause, which are internally prefixed with a `Not` to make it easier to work with.
For example, the negation of `At(obj, loc)` will be input as `~At(obj, loc)` and internally represented as `NotAt(obj, loc)`. 
This equivalently creates a new clause for each negative literal, removing the hassle of maintaining two separate knowledge bases.
This greatly simplifies algorithms like `GraphPlan` as we will see later.
The `convert` method takes an input string, parses it, removes conjunctions if any and returns a list of `Expr` objects.
The `check_precond` method checks if the preconditions for that action are valid, given a `kb`.
The `act` method carries out the action on the given knowledge base.

Now lets try to define a planning problem using these tools. Since we already know about the map of Romania, lets see if we can plan a trip across a simplified map of Romania.

Here is our simplified map definition:

In [4]:
from utils import *
# this imports the required expr so we can create our knowledge base

knowledge_base = [
    expr("Connected(Bucharest,Pitesti)"),
    expr("Connected(Pitesti,Rimnicu)"),
    expr("Connected(Rimnicu,Sibiu)"),
    expr("Connected(Sibiu,Fagaras)"),
    expr("Connected(Fagaras,Bucharest)"),
    expr("Connected(Pitesti,Craiova)"),
    expr("Connected(Craiova,Rimnicu)")
    ]

Let us add some logic propositions to complete our knowledge about travelling around the map. These are the typical symmetry and transitivity properties of connections on a map. We can now be sure that our `knowledge_base` understands what it truly means for two locations to be connected in the sense usually meant by humans when we use the term.

Let's also add our starting location - *Sibiu* to the map.

In [5]:
knowledge_base.extend([
     expr("Connected(x,y) ==> Connected(y,x)"),
     expr("Connected(x,y) & Connected(y,z) ==> Connected(x,z)"),
     expr("At(Sibiu)")
    ])

We now have a complete knowledge base, which can be seen like this:

In [6]:
knowledge_base

[Connected(Bucharest, Pitesti),
 Connected(Pitesti, Rimnicu),
 Connected(Rimnicu, Sibiu),
 Connected(Sibiu, Fagaras),
 Connected(Fagaras, Bucharest),
 Connected(Pitesti, Craiova),
 Connected(Craiova, Rimnicu),
 (Connected(x, y) ==> Connected(y, x)),
 ((Connected(x, y) & Connected(y, z)) ==> Connected(x, z)),
 At(Sibiu)]

We now define possible actions to our problem. We know that we can drive between any connected places. But, as is evident from [this](https://en.wikipedia.org/wiki/List_of_airports_in_Romania) list of Romanian airports, we can also fly directly between Sibiu, Bucharest, and Craiova.

We can define these flight actions like this:

In [7]:
#Sibiu to Bucharest
precond = 'At(Sibiu)'
effect = 'At(Bucharest) & ~At(Sibiu)'
fly_s_b = Action('Fly(Sibiu, Bucharest)', precond, effect)

#Bucharest to Sibiu
precond = 'At(Bucharest)'
effect = 'At(Sibiu) & ~At(Bucharest)'
fly_b_s = Action('Fly(Bucharest, Sibiu)', precond, effect)

#Sibiu to Craiova
precond = 'At(Sibiu)'
effect = 'At(Craiova) & ~At(Sibiu)'
fly_s_c = Action('Fly(Sibiu, Craiova)', precond, effect)

#Craiova to Sibiu
precond = 'At(Craiova)'
effect = 'At(Sibiu) & ~At(Craiova)'
fly_c_s = Action('Fly(Craiova, Sibiu)', precond, effect)

#Bucharest to Craiova
precond = 'At(Bucharest)'
effect = 'At(Craiova) & ~At(Bucharest)'
fly_b_c = Action('Fly(Bucharest, Craiova)', precond, effect)

#Craiova to Bucharest
precond = 'At(Craiova)'
effect = 'At(Bucharest) & ~At(Craiova)'
fly_c_b = Action('Fly(Craiova, Bucharest)', precond, effect)

And the drive actions like this.

In [8]:
#Drive
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
drive = Action('Drive(x, y)', precond, effect)

Our goal is defined as

In [9]:
goals = 'At(Bucharest)'

Finally, we can define a a function that will tell us when we have reached our destination, Bucharest.

In [10]:
def goal_test(kb):
    return kb.ask(expr('At(Bucharest)'))

Thus, with all the components in place, we can define the planning problem.

In [11]:
prob = PDDL(knowledge_base, goals, [fly_s_b, fly_b_s, fly_s_c, fly_c_s, fly_b_c, fly_c_b, drive])

## PLANNING PROBLEMS
---

## Air Cargo Problem

In the Air Cargo problem, we start with cargo at two airports, SFO and JFK. Our goal is to send each cargo to the other airport. We have two airplanes to help us accomplish the task. 
The problem can be defined with three actions: Load, Unload and Fly. 
Let us look how the `air_cargo` problem has been defined in the module. 

In [12]:
psource(air_cargo)

**At(c, a):** The cargo **'c'** is at airport **'a'**.

**~At(c, a):** The cargo **'c'** is _not_ at airport **'a'**.

**In(c, p):** Cargo **'c'** is in plane **'p'**.

**~In(c, p):** Cargo **'c'** is _not_ in plane **'p'**.

**Cargo(c):** Declare **'c'** as cargo.

**Plane(p):** Declare **'p'** as plane.

**Airport(a):** Declare **'a'** as airport.



In the `initial_state`, we have cargo C1, plane P1 at airport SFO and cargo C2, plane P2 at airport JFK. 
Our goal state is to have cargo C1 at airport JFK and cargo C2 at airport SFO. We will discuss on how to achieve this. Let us now define an object of the `air_cargo` problem:

In [13]:
airCargo = air_cargo()

Before taking any actions, we will check if `airCargo` has reached its goal:

In [14]:
print(airCargo.goal_test())

False


It returns False because the goal state is not yet reached. Now, we define the sequence of actions that it should take in order to achieve the goal.
The actions are then carried out on the `airCargo` PDDL.

The actions available to us are the following: Load, Unload, Fly

**Load(c, p, a):** Load cargo **'c'** into plane **'p'** from airport **'a'**.

**Fly(p, f, t):** Fly the plane **'p'** from airport **'f'** to airport **'t'**.

**Unload(c, p, a):** Unload cargo **'c'** from plane **'p'** to airport **'a'**.

This problem can have multiple valid solutions.
One such solution is shown below.

In [15]:
solution = [expr("Load(C1 , P1, SFO)"),
            expr("Fly(P1, SFO, JFK)"),
            expr("Unload(C1, P1, JFK)"),
            expr("Load(C2, P2, JFK)"),
            expr("Fly(P2, JFK, SFO)"),
            expr("Unload (C2, P2, SFO)")] 

for action in solution:
    airCargo.act(action)

As the `airCargo` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal:

In [16]:
print(airCargo.goal_test())

True


It has now achieved its goal.

## The Spare Tire Problem

Let's consider the problem of changing a flat tire of a car. 
The goal is to mount a spare tire onto the car's axle, given that we have a flat tire on the axle and a spare tire in the trunk. 

In [17]:
psource(spare_tire)

**At(obj, loc):** object **'obj'** is at location **'loc'**.

**~At(obj, loc):** object **'obj'** is _not_ at location **'loc'**.

**Tire(t):** Declare a tire of type **'t'**.

Let us now define an object of `spare_tire` problem:

In [18]:
spareTire = spare_tire()

Before taking any actions, we will check if `spare_tire` has reached its goal:

In [19]:
print(spareTire.goal_test())

False


As we can see, it hasn't completed the goal. 
We now define a possible solution that can help us reach the goal of having a spare tire mounted onto the car's axle. 
The actions are then carried out on the `spareTire` PDDL.

The actions available to us are the following: Remove, PutOn

**Remove(obj, loc):** Remove the tire **'obj'** from the location **'loc'**.

**PutOn(t, Axle):** Attach the tire **'t'** on the Axle.

**LeaveOvernight():** We live in a particularly bad neighborhood and all tires, flat or not, are stolen if we leave them overnight.



In [20]:
solution = [expr("Remove(Flat, Axle)"),
            expr("Remove(Spare, Trunk)"),
            expr("PutOn(Spare, Axle)")]

for action in solution:
    spareTire.act(action)

In [21]:
print(spareTire.goal_test())

True


This is a valid solution.
<br>
Another possible solution is

In [22]:
spareTire = spare_tire()

solution = [expr('Remove(Spare, Trunk)'),
            expr('Remove(Flat, Axle)'),
            expr('PutOn(Spare, Axle)')]

for action in solution:
    spareTire.act(action)

In [23]:
print(spareTire.goal_test())

True


Notice that both solutions work, which means that the problem can be solved irrespective of the order in which the `Remove` actions take place, as long as both `Remove` actions take place before the `PutOn` action.

We have successfully mounted a spare tire onto the axle.

## Three Block Tower Problem

This problem's domain consists of a set of cube-shaped blocks sitting on a table. 
The blocks can be stacked, but only one block can fit directly on top of another.
A robot arm can pick up a block and move it to another position, either on the table or on top of another block. 
The arm can pick up only one block at a time, so it cannot pick up a block that has another one on it. 
The goal will always be to build one or more stacks of blocks. 
In our case, we consider only three blocks.
The particular configuration we will use is called the Sussman anomaly after Prof. Gerry Sussman.

Let's take a look at the definition of `three_block_tower()` in the module.

In [24]:
psource(three_block_tower)

**On(b, x):** The block **'b'** is on **'x'**. **'x'** can be a table or a block.

**~On(b, x):** The block **'b'** is _not_ on **'x'**. **'x'** can be a table or a block.

**Block(b):** Declares **'b'** as a block.

**Clear(x):** To indicate that there is nothing on **'x'** and it is free to be moved around.

**~Clear(x):** To indicate that there is something on **'x'** and it cannot be moved.
 
 Let us now define an object of `three_block_tower` problem:

In [25]:
threeBlockTower = three_block_tower()

Before taking any actions, we will check if `threeBlockTower` has reached its goal:

In [26]:
print(threeBlockTower.goal_test())

False


As we can see, it hasn't completed the goal. 
We now define a sequence of actions that can stack three blocks in the required order. 
The actions are then carried out on the `threeBlockTower` PDDL.

The actions available to us are the following: MoveToTable, Move

**MoveToTable(b, x): ** Move box **'b'** stacked on **'x'** to the table, given that box **'b'** is clear.

**Move(b, x, y): ** Move box **'b'** stacked on **'x'** to the top of **'y'**, given that both **'b'** and **'y'** are clear.


In [27]:
solution = [expr("MoveToTable(C, A)"),
            expr("Move(B, Table, C)"),
            expr("Move(A, Table, B)")]

for action in solution:
    threeBlockTower.act(action)

As the `three_block_tower` has taken all the steps it needed in order to achieve the goal, we can now check if it has acheived its goal.

In [28]:
print(threeBlockTower.goal_test())

True


It has now successfully achieved its goal i.e, to build a stack of three blocks in the specified order.

## Shopping Problem

This problem requires us to acquire a carton of milk, a banana and a drill.
Initially, we start from home and it is known to us that milk and bananas are available in the supermarket and the hardware store sells drills.
Let's take a look at the definition of the `shopping_problem` in the module.

In [29]:
psource(shopping_problem)

**At(x):** Indicates that we are currently at **'x'** where **'x'** can be Home, SM (supermarket) or HW (Hardware store).

**~At(x):** Indicates that we are currently _not_ at **'x'**.

**Sells(s, x):** Indicates that item **'x'** can be bought from store **'s'**.

**Have(x):** Indicates that we possess the item **'x'**.

In [30]:
shoppingProblem = shopping_problem()

Let's first check whether the goal state Have(Milk), Have(Banana), Have(Drill) is reached or not.

In [31]:
print(shoppingProblem.goal_test())

False


Let's look at the possible actions

**Buy(x, store):** Buy an item **'x'** from a **'store'** given that the **'store'** sells **'x'**.

**Go(x, y):** Go to destination **'y'** starting from source **'x'**.

We now define a valid solution that will help us reach the goal.
The sequence of actions will then be carried out onto the `shoppingProblem` PDDL.

In [32]:
solution = [expr('Go(Home, SM)'),
            expr('Buy(Milk, SM)'),
            expr('Buy(Banana, SM)'),
            expr('Go(SM, HW)'),
            expr('Buy(Drill, HW)')]

for action in solution:
    shoppingProblem.act(action)

We have taken the steps required to acquire all the stuff we need. 
Let's see if we have reached our goal.

In [33]:
shoppingProblem.goal_test()

True

It has now successfully achieved the goal.

## Have Cake and Eat Cake Too

This problem requires us to reach the state of having a cake and having eaten a cake simlutaneously, given a single cake.
Let's first take a look at the definition of the `have_cake_and_eat_cake_too` problem in the module.

In [34]:
psource(have_cake_and_eat_cake_too)

Since this problem doesn't involve variables, states can be considered similar to symbols in propositional logic.

**Have(Cake):** Declares that we have a **'Cake'**.

**~Have(Cake):** Declares that we _don't_ have a **'Cake'**.

In [35]:
cakeProblem = have_cake_and_eat_cake_too()

First let us check whether the goal state 'Have(Cake)' and 'Eaten(Cake)' are reached or not.

In [36]:
print(cakeProblem.goal_test())

False


Let us look at the possible actions.

**Bake(x):** To bake **' x '**.

**Eat(x):** To eat **' x '**.

We now define a valid solution that can help us reach the goal.
The sequence of actions will then be acted upon the `cakeProblem` PDDL.

In [37]:
solution = [expr("Eat(Cake)"),
            expr("Bake(Cake)")]

for action in solution:
    cakeProblem.act(action)

Now we have made actions to bake the cake and eat the cake. Let us check if we have reached the goal.

In [38]:
print(cakeProblem.goal_test())

True


It has now successfully achieved its goal i.e, to have and eat the cake.

One might wonder if the order of the actions matters for this problem.
Let's see for ourselves.

In [39]:
cakeProblem = have_cake_and_eat_cake_too()

solution = [expr('Bake(Cake)'),
            expr('Eat(Cake)')]

for action in solution:
    cakeProblem.act(action)

Exception: Action 'Bake(Cake)' pre-conditions not satisfied

It raises an exception.
Indeed, according to the problem, we cannot bake a cake if we already have one.
In planning terms, '~Have(Cake)' is a precondition to the action 'Bake(Cake)'.
Hence, this solution is invalid.

## SOLVING PLANNING PROBLEMS
----
### GRAPHPLAN
<br>
The GraphPlan algorithm is a popular method of solving classical planning problems.
Before we get into the details of the algorithm, let's look at a special data structure called **planning graph**, used to give better heuristic estimates and plays a key role in the GraphPlan algorithm.

### Planning Graph
A planning graph is a directed graph organized into levels. 
Each level contains information about the current state of the knowledge base and the possible state-action links to and from that level.
The first level contains the initial state with nodes representing each fluent that holds in that level.
This level has state-action links linking each state to valid actions in that state.
Each action is linked to all its preconditions and its effect states.
Based on these effects, the next level is constructed.
The next level contains similarly structured information about the next state.
In this way, the graph is expanded using state-action links till we reach a state where all the required goals hold true simultaneously.
We can say that we have reached our goal if none of the goal states in the current level are mutually exclusive.
This will be explained in detail later.
<br>
Planning graphs only work for propositional planning problems, hence we need to eliminate all variables by generating all possible substitutions.
<br>
For example, the planning graph of the `have_cake_and_eat_cake_too` problem might look like this
![title](images/cake_graph.jpg)
<br>
The black lines indicate links between states and actions.
<br>
In every planning problem, we are allowed to carry out the `no-op` action, ie, we can choose no action for a particular state.
These are called 'Persistence' actions and are represented in the graph by the small square boxes.
In technical terms, a persistence action has effects same as its preconditions.
This enables us to carry a state to the next level.
<br>
<br>
The gray lines indicate mutual exclusivity.
This means that the actions connected by a gray line cannot be taken together.
Mutual exclusivity (mutex) occurs in the following cases:
1. **Inconsistent effects**: One action negates the effect of the other. For example, _Eat(Cake)_ and the persistence of _Have(Cake)_ have inconsistent effects because they disagree on the effect _Have(Cake)_
2. **Interference**: One of the effects of an action is the negation of a precondition of the other. For example, _Eat(Cake)_ interferes with the persistence of _Have(Cake)_ by negating its precondition.
3. **Competing needs**: One of the preconditions of one action is mutually exclusive with a precondition of the other. For example, _Bake(Cake)_ and _Eat(Cake)_ are mutex because they compete on the value of the _Have(Cake)_ precondition.

In the module, planning graphs have been implemented using two classes, `Level` which stores data for a particular level and `Graph` which connects multiple levels together.
Let's look at the `Level` class.

In [40]:
psource(Level)

Each level stores the following data
1. The current state of the level in `current_state`
2. Links from an action to its preconditions in `current_action_links`
3. Links from a state to the possible actions in that state in `current_state_links`
4. Links from each action to its effects in `next_action_links`
5. Links from each possible next state from each action in `next_state_links`. This stores the same information as the `current_action_links` of the next level.
6. Mutex links in `mutex`.
<br>
<br>
The `find_mutex` method finds the mutex links according to the points given above.
<br>
The `build` method populates the data structures storing the state and action information.
Persistence actions for each clause in the current state are also defined here. 
The newly created persistence action has the same name as its state, prefixed with a 'P'.

Let's now look at the `Graph` class.

In [41]:
psource(Graph)

The class stores a problem definition in `pddl`, 
a knowledge base in `kb`, 
a list of `Level` objects in `levels` and 
all the possible arguments found in the initial state of the problem in `objects`.
<br>
The `expand_graph` method generates a new level of the graph.
This method is invoked when the goal conditions haven't been met in the current level or the actions that lead to it are mutually exclusive.
The `non_mutex_goals` method checks whether the goals in the current state are mutually exclusive.
<br>
<br>
Using these two classes, we can define a planning graph which can either be used to provide reliable heuristics for planning problems or used in the `GraphPlan` algorithm.
<br>
Let's have a look at the `GraphPlan` class.

In [42]:
psource(GraphPlan)

Given a planning problem defined as a PDDL, `GraphPlan` creates a planning graph stored in `graph` and expands it till it reaches a state where all its required goals are present simultaneously without mutual exclusivity.
<br>
Once a goal is found, `extract_solution` is called.
This method recursively finds the path to a solution given a planning graph.
In the case where `extract_solution` fails to find a solution for a set of goals as a given level, we record the `(level, goals)` pair as a **no-good**.
Whenever `extract_solution` is called again with the same level and goals, we can find the recorded no-good and immediately return failure rather than searching again. 
No-goods are also used in the termination test.
<br>
The `check_leveloff` method checks if the planning graph for the problem has **levelled-off**, ie, it has the same states, actions and mutex pairs as the previous level.
If the graph has already levelled off and we haven't found a solution, there is no point expanding the graph, as it won't lead to anything new.
In such a case, we can declare that the planning problem is unsolvable with the given constraints.
<br>
<br>
To summarize, the `GraphPlan` algorithm calls `expand_graph` and tests whether it has reached the goal and if the goals are non-mutex.
<br>
If so, `extract_solution` is invoked which recursively reconstructs the solution from the planning graph.
<br>
If not, then we check if our graph has levelled off and continue if it hasn't.

Let's solve a few planning problems that we had defined earlier.

Air cargo problem:
<br>
In accordance with the summary above, we have defined a helper function to carry out `GraphPlan` on the `air_cargo` problem.
The function is pretty straightforward.
Let's have a look.

In [43]:
psource(air_cargo_graphplan)

Let's instantiate the problem and find a solution using this helper function.

In [44]:
air_cargo = air_cargo_graphplan()
air_cargo

[[[PCargo(C2),
   Load(C2, P2, JFK),
   PPlane(P2),
   Load(C1, P1, SFO),
   Fly(P1, SFO, JFK),
   PAirport(SFO),
   PAirport(JFK),
   PPlane(P1),
   PCargo(C1),
   Fly(P2, JFK, SFO)],
  [Unload(C2, P2, SFO), Unload(C1, P1, JFK)]]]

Each element in the solution is a valid action.
The solution is separated into lists for each level.
The actions prefixed with a 'P' are persistence actions and can be ignored.
They simply carry certain states forward.
We have another helper function `linearize` that presents the solution in a more readable format, much like a total-order planner.

In [45]:
linearize(air_cargo)

[Load(C2, P2, JFK),
 Load(C1, P1, SFO),
 Fly(P1, SFO, JFK),
 Fly(P2, JFK, SFO),
 Unload(C2, P2, SFO),
 Unload(C1, P1, JFK)]

Indeed, this is a correct solution.
<br>
There are similar helper functions for some other planning problems.
<br>
Lets' try solving the spare tire problem.

In [46]:
spare_tire = spare_tire_graphplan()
linearize(spare_tire)

[Remove(Flat, Axle), Remove(Spare, Trunk), PutOn(Spare, Axle)]

Solution for the cake problem

In [47]:
cake_problem = have_cake_and_eat_cake_too_graphplan()
linearize(cake_problem)

[Eat(Cake), Bake(Cake)]

Solution for the Sussman's Anomaly configuration of three blocks.

In [48]:
sussman_anomaly = three_block_tower_graphplan()
linearize(sussman_anomaly)

[MoveToTable(C, A), Move(B, Table, C), Move(A, Table, B)]